In [1]:
%autosave 0

Autosave disabled


In [ ]:
Many imports! we require the new TFIDvectorizer

In [9]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from env import get_connection
from prepare import clean, lemmatize

Defining a url and query hat we can use to retrieve data from Codeup's sql server

In [10]:
url = get_connection('spam_db')

query = 'SELECT * FROM spam'

df = pd.read_sql(query, url, index_col='id')

df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We can create clean version of our dataset by applying the clean function to our text data 

In [11]:
df['clean_text'] = df.text.apply(clean, args=['us'])
df.head()

,label,text,clean_text
id,,,
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah ' think goes usf lives around though


In [ ]:
I choose to apply lemmatization because the dataset a

In [12]:
df['lemmas'] = df.clean_text.apply(lemmatize)
df.head()

,label,text,clean_text,lemmas
id,,,,
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif oni,ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah ' think goes usf lives around though,nah ' think go usf life around though


We will create our x values from the lemmas and our y values from the labels for the observations

In [16]:
X = df.lemmas
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=42)

we have to create an instance of tdidfvectorizer we will use it in a similar way to most sklearn objects.

In [14]:
tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf[:10]

<10x7604 sparse matrix of type '<class 'numpy.float64'>'
	with 63 stored elements in Compressed Sparse Row format>

we have generated over 7600 features from our dataset we can feed these feature into a machiene learning model and generates prediction

In [18]:
lm = LogisticRegression()

lm.fit(X_train_tfidf, y_train)


LogisticRegression()

In [19]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': lm.predict(X_train_tfidf)})
y_train_res.head()

,actual,preds
id,,
708,spam,spam
4338,ham,ham
5029,ham,ham
4921,ham,ham
2592,ham,ham


In [20]:
y_train.shape

(3900,)

In [21]:
print(classification_report(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      3372
        spam       0.99      0.73      0.84       528

    accuracy                           0.96      3900
   macro avg       0.98      0.86      0.91      3900
weighted avg       0.96      0.96      0.96      3900



In [24]:
y_train_res = pd.DataFrame({'actual': y_test,
                            'preds': lm.predict(X_test_tfidf)})
y_train_res.head()

,actual,preds
id,,
3245,ham,ham
944,ham,ham
1044,spam,ham
2484,ham,ham
812,spam,spam


In [28]:
classification_report(y_train_res.actual, y_train_res.preds)

'              precision    recall  f1-score   support\n\n         ham       0.95      1.00      0.97      1453\n        spam       0.95      0.63      0.76       219\n\n    accuracy                           0.95      1672\n   macro avg       0.95      0.81      0.86      1672\nweighted avg       0.95      0.95      0.94      1672\n'

In [27]:
print(classification_report(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97      1453
        spam       0.95      0.63      0.76       219

    accuracy                           0.95      1672
   macro avg       0.95      0.81      0.86      1672
weighted avg       0.95      0.95      0.94      1672

